In [4]:
import tempfile
import shutil
import os
import unittest
import pytest
import logging
import warnings
import torch
from deeprankcore.Trainer import Trainer
from deeprankcore.DataSet import HDF5DataSet

from deeprankcore.ginet import GINet
from deeprankcore.foutnet import FoutNet
from deeprankcore.naive_gnn import NaiveNetwork
from deeprankcore.sGAT import sGAT
from deeprankcore.models.metrics import (
    OutputExporter,
    TensorboardBinaryClassificationExporter,
    ScatterPlotExporter
)
from deeprankcore.domain.features import groups, edgefeats
from deeprankcore.domain.features import nodefeats as Nfeat
from deeprankcore.domain import targettypes as targets

default_features = [Nfeat.RESTYPE, Nfeat.POLARITY, Nfeat.BSA, Nfeat.RESDEPTH, Nfeat.HSE, Nfeat.INFOCONTENT, Nfeat.PSSM]


In [10]:
# old

def _model_base_test( # pylint: disable=too-many-arguments, too-many-locals
    train_hdf5_path,
    val_hdf5_path,
    test_hdf5_path,
    model_class,
    node_features,
    edge_features,
    task,
    target,
    metrics_exporters,
    transform_sigmoid,
    clustering_method,
    use_cuda = False
):

    dataset_train = HDF5DataSet(
        hdf5_path=train_hdf5_path,
        root="./",
        node_feature=node_features,
        edge_feature=edge_features,
        task = task,
        target=target,
        clustering_method=clustering_method)

    if val_hdf5_path is not None:
        dataset_val = HDF5DataSet(
            hdf5_path=val_hdf5_path,
            root="./",
            node_feature=node_features,
            edge_feature=edge_features,
            task = task,
            target=target,
            clustering_method=clustering_method)
    else:
        dataset_val = None

    if test_hdf5_path is not None:
        dataset_test = HDF5DataSet(
            hdf5_path=test_hdf5_path,
            root="./",
            node_feature=node_features,
            edge_feature=edge_features,
            target=target,
            task=task,
            clustering_method=clustering_method)
    else:
        dataset_test = None

    trainer = Trainer(
        dataset_train,
        dataset_val,
        dataset_test,
        model_class,
        batch_size=64,
        metrics_exporters=metrics_exporters,
        transform_sigmoid=transform_sigmoid,
    )

    trainer.train(nepoch=10, validate=True)

    trainer.save_model("test.pth.tar")

    Trainer(
        dataset_train,
        dataset_val,
        dataset_test,
        model_class,
        pretrained_model="test.pth.tar")



In [13]:
_model_base_test(
    "./data/hdf5/1ATN_ppi.hdf5",
    "./data/hdf5/1ATN_ppi.hdf5",
    "./data/hdf5/1ATN_ppi.hdf5",
    GINet,
    default_features,
    [edgefeats.DISTANCE],
    targets.REGRESS,
    targets.IRMSD,
    [OutputExporter('/home/dbodor/git/DeepRank/deeprank-core/metrics/notebooks')],
    True,
    "mcl",
)

   ['./data/hdf5/1ATN_ppi.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 287.95it/s, mol=1ATN_ppi.hdf5]
   ['./data/hdf5/1ATN_ppi.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 271.81it/s, mol=1ATN_ppi.hdf5]
   ['./data/hdf5/1ATN_ppi.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 352.73it/s, mol=1ATN_ppi.hdf5]


100%|██████████| 4/4 [00:00<00:00,  9.51it/s]


OSError: directory ``/home/dbodor/git/DeepRank/deeprank-core/tests/metrics`` exists but it can not be written